In [3]:
import pandas as pd
from config import database_url

In [4]:
    sel = pd.read_sql_table('predicted_table', database_url)
    sel = sel.drop(sel.columns[0], axis=1)

    sel['Date'] = sel['Date'].dt.strftime('%Y-%m-%d')

          Date  Instant_food_products_Predicted Instant_food_products  \
0   2014-01-01                              0.0                   2.0   
1   2014-01-02                              0.0                   0.0   
2   2014-01-03                              0.0                   0.0   
3   2014-01-04                              0.0                   0.0   
4   2014-01-05                              0.0                   0.0   
..         ...                              ...                   ...   
936 2016-07-26                              0.0                         
937 2016-07-27                              0.0                         
938 2016-07-28                              0.0                         
939 2016-07-29                              0.0                         
940 2016-07-30                              0.0                         

     UHT-milk_Predicted UHT-milk  abrasive_cleaner_Predicted abrasive_cleaner  \
0                   0.0      0.0          

In [22]:
sum_price = sel['whole_milk'].sum()
print(sum_price)
# Convert the entire dataframe data types into numeric for calculations
calc_df = sel.apply(pd.to_numeric, errors='ignore')
calc_df.info()


2.03.01.09.01.07.05.015.02.06.02.02.01.08.04.07.02.01.03.02.05.05.04.06.04.03.03.00.06.08.04.03.02.04.02.01.05.02.04.02.01.04.04.01.03.04.08.05.02.08.02.05.05.02.05.05.05.03.03.02.01.04.02.05.07.05.01.02.03.04.06.06.00.02.07.05.03.02.04.03.00.09.04.02.00.04.03.07.02.05.05.07.02.02.00.04.05.06.04.03.01.05.04.04.03.05.04.05.05.06.01.05.01.02.02.05.01.02.02.04.02.03.04.07.03.03.01.04.03.02.05.09.01.05.03.06.04.02.04.04.00.07.03.01.09.04.03.08.03.02.01.04.02.01.05.06.07.01.01.03.06.05.03.05.05.06.02.03.04.01.03.03.02.03.04.04.05.02.03.01.03.05.02.01.04.04.04.04.03.03.01.09.01.06.03.02.02.06.02.06.02.01.05.01.05.02.01.03.01.03.03.00.07.05.02.04.00.03.01.04.01.04.01.06.01.03.04.07.03.02.02.03.03.06.04.06.05.05.06.04.05.03.03.05.04.07.04.04.05.07.00.01.03.01.04.03.04.08.00.06.01.04.04.02.02.05.03.04.00.04.03.02.01.02.04.03.05.04.04.03.02.04.02.03.02.02.02.03.02.05.00.03.01.06.03.06.02.06.04.02.07.07.04.07.02.03.06.05.02.04.03.04.02.03.03.06.03.05.01.03.04.06.04.05.03.05.02.07.04.02.02.08.02.0

In [23]:
sum_quantity = calc_df['whole_milk'].sum()
min_quantity 
print(sum_price)

2502.0